In [ ]:
!pip install googletrans==4.0.0-rc1
!pip install deepl

In [ ]:
class Logger:
  def __init__(self, filename, output=False):
    self.console = sys.stdout
    self.output = output
    self.file = None
    self.set_file(filename)

  def set_file(self, filename):
    if self.file:
      self.file.flush()
      self.file.close()
    self.file = open(filename, 'w')

  def write(self, message):
    if self.output:
      self.console.write(str(message) + '\n')
    if self.file:
      self.file.write(str(message) + '\n')

  def flush(self):
    self.console.flush()
    if self.file: 
      self.file.flush()

# Read Test Data

In [ ]:
import csv
import sys
import nltk

MAX_ROWS = 50

# reference original Czech texts
reference_originals = []

# reference translations done by humans in English
reference_translations = []

csv.field_size_limit(sys.maxsize)

try:
  # Czech parallel corpus https://ufal.mff.cuni.cz/czeng
  with open('/content/drive/MyDrive/Colab Notebooks/VM2/data/translation/czeng20-test', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter='\t', quotechar='|')
    counter = 0

    # skip lines
    for _ in range(50):
        next(reader)

    while reader:
      row = next(reader)
      if row and len(row) == 6:
        reference_originals.append(row[4])
        reference_translations.append(row[5])
        counter = counter + 1
      
      if counter >= MAX_ROWS:
        break;
except StopIteration:
  print()

logger = Logger("translation_overview.csv", output=True)

In [ ]:
print("Number of Translations: " + len(reference_translations))

50


# Translate Test Data using Google Translate

In [ ]:
from googletrans import Translator
translator = Translator()

# resulting translations from Czech to English
hypothesis_translations_google = []
for original in reference_originals:
  result = translator.translate(original, dest="EN", src="CS")
  hypothesis_translations_google.append(result.text)

BLEUScore = 0
for i in range(len(hypothesis_translations_google)):
  BLEUScore += nltk.translate.bleu_score.sentence_bleu([reference_translations[i]], hypothesis_translations_google[i])

BLEUScore = BLEUScore/len(hypothesis_translations_google)
print("Google Translate BLEU Score " + str(BLEUScore))

Google Translate BLEU Score 0.5746241870747856


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


# Translate Test Data using DeepL

In [ ]:
import deepl
translator = deepl.Translator("<key>")

# resulting translations from Czech to English
hypothesis_translations_deepl = []
for original in reference_originals:
  result = translator.translate_text(original, target_lang="EN-GB", source_lang="CS")
  hypothesis_translations_deepl.append(result.text)

BLEUScore = 0
for i in range(len(hypothesis_translations_deepl)):
  BLEUScore += nltk.translate.bleu_score.sentence_bleu([reference_translations[i]], hypothesis_translations_deepl[i])

BLEUScore = BLEUScore/len(hypothesis_translations_deepl)
print("DeepL BLEU Score " + str(BLEUScore))

DeepL BLEU Score 0.6608219505348069


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


# Translate Test Data using Microsoft Cognitive Services


In [ ]:
import requests, uuid, json

subscription_key = "<key>"
endpoint = "https://api.cognitive.microsofttranslator.com"
location = "switzerlandwest"
path = '/translate'
constructed_url = endpoint + path
params = {
    'api-version': '3.0',
    'from': 'CS',
    'to': ['EN']
}
headers = {
    'Ocp-Apim-Subscription-Key': subscription_key,
    'Ocp-Apim-Subscription-Region': location,
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())
}

body = []
for original in reference_originals:
  body.append({ 'text': original })

# execute request
request = requests.post(constructed_url, params=params, headers=headers, json=body)
responses = request.json()

# resulting translations from Czech to English
hypothesis_translations_microsoft = []
for response in responses: 
  hypothesis_translations_microsoft.append(response["translations"][0]["text"])

BLEUScore = 0
for i in range(len(hypothesis_translations_microsoft)):
  BLEUScore += nltk.translate.bleu_score.sentence_bleu([reference_translations[i]], hypothesis_translations_microsoft[i])

BLEUScore = BLEUScore/len(hypothesis_translations_microsoft)
print("Microsoft Translate BLEU Score " + str(BLEUScore))

Microsoft Translate BLEU Score 0.6374220560930838


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


# Prepare data (translation and ground truth) for black and whitebox judegement

In [ ]:
logger.write("Original_English_Translation; Google_English_Translation; Deepl_English_Translation; Microsoft_English_Tranlsation")

for i in range(len(reference_translations)):
  output = reference_translations[i]

  output = output + "; " + hypothesis_translations_google[i]
  output = output + "; " + hypothesis_translations_deepl[i]
  output = output + "; " + hypothesis_translations_microsoft[i]

  logger.write(output)

logger.flush()

Original_English_Translation; Google_English_Translation; Deepl_English_Translation; Microsoft_English_Tranlsation
It's... sacred Navajo herbs and...; That's ... secret Navajské flowers and ...; It's... secret Navajo flowers and...; That is... secret Navajo flowers and...
Hey, is everything okay?; Look, is everything all right?; Hey, is everything okay?; Hey, is everything okay?
No.; No.; No.; No.
Not really.; Not exactly.; Not really.; Not really.
Actually, not at all.; Actually, it's not at all.; Actually, it's not at all.; Actually, not at all.
Subject: Application of EU legislation Answer from the Council (3 November 2009); Subject: EU legislation legislation Response (November 3, 2009); Subject: Application of EU law Answer from the Council (3 November 2009); Subject: Application of EU law Council reply (3 November 2009)
It was to be a test...; It was just a test ...; It was just supposed to be a test...; It was supposed to be just a test...
Samah gazed at Xar in earnest appeal.; 